In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,62,2024-03-05,3,111.491803,96.754098,0.541852,12.640984,26.004918,0.226115,114.565574,...,11.847541,21.763934,0.168000,110.411475,0,3,1,2,22,2024
1,61,2024-03-05,11,121.283333,97.286667,0.578883,10.523333,24.301667,0.196817,123.818333,...,11.598333,24.131667,0.185083,117.080000,0,3,1,2,2,2024
2,62,2024-03-05,1,116.245902,97.896721,0.557689,11.463934,25.583607,0.209705,118.611475,...,11.339344,25.100000,0.184672,115.239344,0,3,1,4,19,2024
3,61,2024-03-05,-2,112.100000,99.575000,0.534967,13.015000,24.653333,0.195200,112.078333,...,11.308333,22.220000,0.222017,115.025000,0,3,1,2,9,2024
4,61,2024-03-05,2,117.133333,97.738333,0.535617,9.950000,24.925000,0.238467,119.496667,...,10.701667,24.776667,0.174333,115.206667,0,3,1,2,23,2024
5,61,2024-03-05,-2,120.433333,100.435000,0.537883,10.836667,27.643333,0.216800,119.025000,...,11.039344,29.522951,0.201852,118.972131,0,3,1,3,1,2024
6,62,2024-03-05,2,112.344262,100.945902,0.532033,12.577049,22.695082,0.172115,111.047541,...,11.175000,24.688333,0.205567,113.430000,0,3,1,2,27,2024
7,63,2024-03-05,-2,123.258065,101.574194,0.580355,11.177419,23.724194,0.176532,121.340323,...,10.657377,22.173770,0.198393,119.018033,0,3,1,2,12,2024
8,62,2024-03-05,-2,117.098361,98.172131,0.564475,12.732787,24.842623,0.240836,118.863934,...,10.801639,26.016393,0.176623,119.329508,0,3,1,2,24,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-05,ORL,CHO,1.266667,3.950000,1
1,2024-03-05,BOS,CLE,1.294118,3.700000,1
2,2024-03-05,NOP,TOR,1.235294,4.300000,1
3,2024-03-05,DET,MIA,5.550000,1.160000,0
4,2024-03-05,PHI,BRK,1.925926,1.892857,1
5,2024-03-05,ATL,NYK,2.240000,1.675676,0
6,2024-03-05,SAS,HOU,3.600000,1.307692,0
7,2024-03-05,IND,DAL,2.540000,1.540541,0
8,2024-03-05,PHO,DEN,4.400000,1.227273,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-05,ORL,CHO,1.266667,3.950000,1
1,2024-03-05,BOS,CLE,1.294118,3.700000,0
2,2024-03-05,NOP,TOR,1.235294,4.300000,0
3,2024-03-05,DET,MIA,5.550000,1.160000,0
4,2024-03-05,PHI,BRK,1.925926,1.892857,0
5,2024-03-05,ATL,NYK,2.240000,1.675676,0
6,2024-03-05,SAS,HOU,3.600000,1.307692,0
7,2024-03-05,IND,DAL,2.540000,1.540541,0
8,2024-03-05,PHO,DEN,4.400000,1.227273,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-05,ORL,CHO,1.266667,3.950000,1
1,2024-03-05,BOS,CLE,1.294118,3.700000,0
2,2024-03-05,NOP,TOR,1.235294,4.300000,0
3,2024-03-05,DET,MIA,5.550000,1.160000,0
4,2024-03-05,PHI,BRK,1.925926,1.892857,0
5,2024-03-05,ATL,NYK,2.240000,1.675676,0
6,2024-03-05,SAS,HOU,3.600000,1.307692,0
7,2024-03-05,IND,DAL,2.540000,1.540541,0
8,2024-03-05,PHO,DEN,4.400000,1.227273,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-05,ORL,CHO,1.266667,3.950000,1
1,2024-03-05,BOS,CLE,1.294118,3.700000,1
2,2024-03-05,NOP,TOR,1.235294,4.300000,1
3,2024-03-05,DET,MIA,5.550000,1.160000,0
4,2024-03-05,PHI,BRK,1.925926,1.892857,0
5,2024-03-05,ATL,NYK,2.240000,1.675676,0
6,2024-03-05,SAS,HOU,3.600000,1.307692,0
7,2024-03-05,IND,DAL,2.540000,1.540541,0
8,2024-03-05,PHO,DEN,4.400000,1.227273,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-05,ORL,CHO,1.266667,3.950000,1.0
1,2024-03-05,BOS,CLE,1.294118,3.700000,1.0
2,2024-03-05,NOP,TOR,1.235294,4.300000,1.0
3,2024-03-05,DET,MIA,5.550000,1.160000,0.0
4,2024-03-05,PHI,BRK,1.925926,1.892857,1.0
5,2024-03-05,ATL,NYK,2.240000,1.675676,1.0
6,2024-03-05,SAS,HOU,3.600000,1.307692,1.0
7,2024-03-05,IND,DAL,2.540000,1.540541,1.0
8,2024-03-05,PHO,DEN,4.400000,1.227273,0.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-05,ORL,CHO,1.266667,3.950000,1.0
1,2024-03-05,BOS,CLE,1.294118,3.700000,1.0
2,2024-03-05,NOP,TOR,1.235294,4.300000,1.0
3,2024-03-05,DET,MIA,5.550000,1.160000,0.0
4,2024-03-05,PHI,BRK,1.925926,1.892857,1.0
5,2024-03-05,ATL,NYK,2.240000,1.675676,1.0
6,2024-03-05,SAS,HOU,3.600000,1.307692,1.0
7,2024-03-05,IND,DAL,2.540000,1.540541,1.0
8,2024-03-05,PHO,DEN,4.400000,1.227273,0.0
